In [1]:
import cv2 
from matplotlib import pyplot as plt
import numpy as np
import math
import time



from pynq import Overlay
ol = Overlay('/home/xilinx/jupyter_notebooks/joshi_final_project/dma_axis_ip_example.bit')

In [2]:
ol?
ol.ip_dict

{'gaussianBlur_0': {'type': 'xilinx.com:hls:gaussianBlur:1.0',
  'mem_id': 's_axi_control',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {},
  'parameters': {'C_S_AXI_CONTROL_ADDR_WIDTH': '4',
   'C_S_AXI_CONTROL_DATA_WIDTH': '32',
   'Component_Name': 'dma_axis_ip_example_gaussianBlur_0_0',
   'clk_period': '10',
   'machine': '64',
   'combinational': '0',
   'latency': '58',
   'II': 'x',
   'EDK_IPTYPE': 'PERIPHERAL',
   'C_S_AXI_CONTROL_BASEADDR': '0x40000000',
   'C_S_AXI_CONTROL_HIGHADDR': '0x4000FFFF',
   'ADDR_WIDTH': '4',
   'DATA_WIDTH': '32',
   'PROTOCOL': 'AXI4LITE',
   'READ_WRITE_MODE': 'READ_WRITE',
   'FREQ_HZ': '100000000',
   'ID_WIDTH': '0',
   'AWUSER_WIDTH': '0',
   'ARUSER_WIDTH': '0',
   'WUSER_WIDTH': '0',
   'RUSER_WIDTH': '0',
   'BUSER_WIDTH': '0',
   'HAS_BURST': '0',
   'HAS_LOCK': '0',
   'HAS_PROT': '0',
   'HAS_CACHE': '0',
   'HAS_QOS': '0',
   'HAS_REGION': '0',
   'HAS_WSTRB': '1',
   'HAS_BRESP': '1',
   'HAS_RRESP': '1',
   'SUPPORTS_NAR

In [3]:
ol.gaussianBlur_0?

In [4]:
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

In [5]:
hls_ip = ol.gaussianBlur_0
CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x81)
hls_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [6]:
# Creates the kernels
def getGaussianKernels(size, sigma = 1):
    kernel = [[0] * size for _ in range(size)]
    center = size // 2

    total = 0
    for x in range(size):
        for y in range(size):
            kernel[x][y] = math.exp(-((x - center)**2 + (y - center)**2) / (2 * sigma**2))
            total += kernel[x][y]

    # Normalize the kernel
    for x in range(size):
        for y in range(size):
            kernel[x][y] /= total

    print(kernel)

    return kernel

In [7]:
def importImages(name):

    image_info = {}
    
    # original_image = cv2.imread('cameraman.jpg')
    noisy_image = cv2.imread(name)


    # cv2.imshow('Image1', original_image)
#     cv2.imshow('Input', noisy_image)

    height, width, channels = noisy_image.shape

    cv2.waitKey(0)
    cv2.destroyAllWindows()


    image_info['height'] = height
    image_info['width'] = width
    image_info['channels'] = channels
    image_info['image'] = noisy_image # list of list image values


    noisy_image = cv2.cvtColor(noisy_image, cv2.COLOR_BGR2GRAY)
    image_info['bw_image'] = noisy_image #single string image
    
    return image_info


In [8]:
def buildImage(pixel_values, dimension, original_image, bw_image):

    image_array = np.array(pixel_values).reshape((dimension, dimension))

    image_array = np.uint8(image_array)

    # Create the image using cv2
    final_image = cv2.resize(image_array, (dimension, dimension), interpolation=cv2.INTER_NEAREST)

    # Display the image
    # cv2.imshow('original input', original_image)
#     cv2.imshow('black and white input', bw_image)
#     cv2.imshow('Output', final_image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    return final_image

In [9]:
#print images: 
def printImages(original_image, final_image): 
    
    plt.figure(figsize=(12, 12))  # Set the size of the figure

    plt.subplot(1, 2, 1)  # 1 row, 2 columns, plot number 1
    plt.imshow(original_image, cmap='gray')
    plt.axis('off')  # Turn off axis
    plt.title('Original Image')
    
    plt.subplot(1, 2, 2)  # 1 row, 2 columns, plot number 2
    plt.imshow(final_image, cmap='gray')
    plt.axis('off')  # Turn off axis
    plt.title('Final Image')
    

In [10]:
def send2Hardware(window):
    #converting window mat into string
    
    dma_arr = []
    for i in range(len(window)):
        for j in range(len(window)):
            dma_arr.append(window[i][j])
    
    data_size = len(window) * len(window)
    input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
    for i in range(data_size):
        input_buffer[i] = dma_arr[i] 
        
    output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
    dma_send.transfer(input_buffer)
    dma_recv.transfer(output_buffer)
    
    sum = output_buffer[0]
    
    del input_buffer, output_buffer
    
    print(sum)
    return sum



# output_lst = []
# data_size = 100
# input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
# for i in range(data_size):
#     input_buffer[i] = example_lst[i]
# output_buffer = allocate(shape=(data_size,), dtype=np.uint32)
# start = time.time()
# dma_send.transfer(input_buffer)
# dma_recv.transfer(output_buffer)
# stop = time.time()

# for i in range(data_size):
#     output_lst.append(output_buffer[i])

# del input_buffer, output_buffer

# axi_stream_time = str((stop - start) * 1000)

In [11]:
#hardware implementation using set multiplication method. 

def gaussianBlur(image, kernel):
    
    down = 0 #determines what the kernel can compute. 
    up = 0
    right = 0 
    left = 0
    kernel_center = int((len(kernel) - 1)/2)
    
    window = []
    for i in range(len(kernel)):
        row = []
        
        for j in range(len(kernel)):
            row.append(0)

        window.append(row)
    
    new_image = []
    for row in range(len(image)):
        
        new_row = []
        
        for col in range(len(image[row])):
            
            # reinitializes the value of the sum to 0
            sum = 0
            
            #center
            window[kernel_center][kernel_center] = image[row][col]

            # check down
            if((row + 1) >= len(image)):
                down = 0 
            else:
                for i in range(kernel_center, 0-1, -1):
                    if (row + i < len(image)):
                        down = i
                        break
                for i in range(down):
                    window[kernel_center + i + 1][kernel_center] = image[row + i + 1][col]
                

            # check up
            if((row - 1) < 0):
                up = 0
            else:
                for i in range(kernel_center, 0-1, -1):
                    if (row - i >= 0):
                        up = i
                        break
                for i in range(up):
                    window[kernel_center - i - 1][kernel_center] = image[row - i - 1][col]


            # check right
            if((col+1) >= len(image[row])):
                right = 0
            else:
                for i in range(kernel_center, 0-1, -1):
                    if (col + i < len(image[row])):
                        right = i
                        break
                for i in range(right):
                    window[kernel_center][kernel_center + i + 1] = image[row][col + i + 1]


            # check left
            if((col-1) < 0):
                left = 0
            else:
                for i in range(kernel_center, 0-1, -1):
                    if((col - i) >= 0):
                        left = i
                        break
                for i in range(left):
                    window[kernel_center][kernel_center - i - 1] = image[row][col - i - 1]


            #bottom right square 
            if (down > 0) and (right > 0):
                for i in range(down):
                    for j in range(right):
                        window[kernel_center + i + 1][kernel_center + j + 1] = image[row + i + 1][col + j + 1]


            #bottom left square 
            if (down > 0) and (left > 0):
                for i in range(down):
                    for j in range(left):
                        window[kernel_center + i + 1][kernel_center - j - 1] = image[row + i + 1][col - j - 1]


            #top right square 
            if (up > 0) and (right > 0):
                for i in range(up):
                    for j in range(right):
                        window[kernel_center - i - 1][kernel_center + j + 1] = image[row - i - 1][col + j + 1]


            #top left square 
            if (up > 0) and (left > 0):
                for i in range(up):
                    for j in range(left):
                        window[kernel_center - i - 1][kernel_center - j - 1] = image[row - i - 1][col - j - 1]


            
            new_row.append(send2Hardware(window)) 

#             print("pixel " + str(row) + " " + str(col) + " complete")


        new_image.append(new_row) # per row
        del new_row


    return new_image

In [12]:
# grabs images
image_info = importImages('barb_noisy.jpg')

In [13]:
# gets the gaussian kernel
image_info['kernel'] = getGaussianKernels(3, 1) # increasing the size of the kernel from 5 to 7 will drastically increase time

[[0.0751136079541115, 0.12384140315297394, 0.0751136079541115], [0.12384140315297394, 0.20417995557165805, 0.12384140315297394], [0.0751136079541115, 0.12384140315297394, 0.0751136079541115]]


In [14]:
from pynq import allocate

# blurs the image (This part is to be converted into HLS
start = time.time()
image_info['gaussian_blur'] = gaussianBlur(image_info['bw_image'], image_info['kernel'])
stop = time.time()

69
pixel 0 0 complete
100
pixel 0 1 complete
118
pixel 0 2 complete
134
pixel 0 3 complete
127
pixel 0 4 complete
108
pixel 0 5 complete
95
pixel 0 6 complete
84
pixel 0 7 complete
76
pixel 0 8 complete
76
pixel 0 9 complete
80
pixel 0 10 complete
80
pixel 0 11 complete
77
pixel 0 12 complete
70
pixel 0 13 complete
65
pixel 0 14 complete
61
pixel 0 15 complete
50
pixel 0 16 complete
47
pixel 0 17 complete
56
pixel 0 18 complete
59
pixel 0 19 complete
56
pixel 0 20 complete
55
pixel 0 21 complete
56
pixel 0 22 complete
58
pixel 0 23 complete
63
pixel 0 24 complete
59
pixel 0 25 complete
57
pixel 0 26 complete
59
pixel 0 27 complete
63
pixel 0 28 complete
73
pixel 0 29 complete
89
pixel 0 30 complete
105
pixel 0 31 complete
111
pixel 0 32 complete
116
pixel 0 33 complete
114
pixel 0 34 complete
110
pixel 0 35 complete
112
pixel 0 36 complete
121
pixel 0 37 complete
119
pixel 0 38 complete
119
pixel 0 39 complete
129
pixel 0 40 complete
135
pixel 0 41 complete
124
pixel 0 42 complete
118


KeyboardInterrupt: 

In [ ]:
# rebuilds the final image
image_info['final_image'] = buildImage(image_info['gaussian_blur'], image_info['height'], image_info['image'], image_info['bw_image'])

In [ ]:
#output the images in matplotlib
printImages(image_info['bw_image'], image_info['final_image'])
print("Execution time for PS: " + str((stop - start)) + " seconds using a 3x3 gaussian kernel")